In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import joblib

In [2]:
import os
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [74]:

file_path_train = '/content/drive/My Drive/lt_train-2.csv'


train_df = pd.read_csv(file_path_train)

In [75]:

file_path_test = '/content/drive/My Drive/lt_test-2.csv'

test_df = pd.read_csv(file_path_test)

In [105]:
train_df

,period_dt,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,location_id,product_id,id
0,2019-04-08,0.0,1049.00,1049.0,9988,1.0,0.0,10,203,203,NaN,425,19474,0
1,2017-05-08,0.0,474.50,949.0,9988,0.0,0.0,10,189,189,1.016438,453,19474,1
2,2017-05-15,0.0,474.50,949.0,9988,0.0,0.0,10,189,189,0.019178,453,19474,2
3,2017-05-22,0.0,474.50,949.0,9988,0.0,0.0,10,189,189,0.019178,453,19474,3
4,2017-05-29,0.0,474.50,949.0,9988,1.0,0.0,10,189,189,0.016438,453,19474,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550139,2019-12-23,0.0,699.50,1399.0,113176,1.0,0.0,10,203,203,NaN,1326,148464,550139
550140,2019-12-30,0.0,699.50,1399.0,113176,1.0,0.0,10,203,203,NaN,1326,148464,550140
550141,2019-12-23,0.0,699.50,1399.0,113176,1.0,0.0,10,219,219,NaN,1380,148464,550141
550142,2019-12-30,0.0,699.50,1399.0,113176,1.0,0.0,10,219,219,NaN,1380,148464,550142


## Features Ing

In [76]:
def get_season(date):
    if date.month in [12, 1, 2]:
        return 'Winter'
    elif date.month in [3, 4, 5]:
        return 'Spring'
    elif date.month in [6, 7, 8]:
        return 'Summer'
    elif date.month in [9, 10, 11]:
        return 'Fall'

In [77]:
train_df['period_dt'] = pd.to_datetime(train_df['period_dt'])
test_df['period_dt'] = pd.to_datetime(test_df['period_dt'])

In [78]:
train_df['year'] = train_df['period_dt'].dt.year
train_df['month'] = train_df['period_dt'].dt.month
train_df['day'] = train_df['period_dt'].dt.day

In [79]:
test_df['year'] = test_df['period_dt'].dt.year
test_df['month'] = test_df['period_dt'].dt.month
test_df['day'] = test_df['period_dt'].dt.day

In [80]:
train_df['season'] = train_df['period_dt'].apply(get_season)
test_df['season'] = test_df['period_dt'].apply(get_season)

In [81]:
train_df = pd.get_dummies(train_df, columns=['season'], drop_first=1)
test_df = pd.get_dummies(test_df, columns=['season'], drop_first=1)

In [82]:
for col in set(train_df.columns) - set(test_df.columns):
    if col.startswith('season_'):
        test_df[col] = 0

In [83]:
for col in set(test_df.columns) - set(train_df.columns):
    if col.startswith('season_'):
        train_df[col] = 0

In [84]:
train_df['season_Spring'] = train_df['season_Spring'].astype(int)
train_df['season_Summer'] = train_df['season_Summer'].astype(int)
train_df['season_Winter'] = train_df['season_Winter'].astype(int)

In [85]:
test_df['season_Spring'] = test_df['season_Spring'].astype(int)
test_df['season_Summer'] = test_df['season_Summer'].astype(int)
test_df['season_Winter'] = test_df['season_Winter'].astype(int)

In [86]:
train_df["weekday"] = train_df.period_dt.dt.weekday
train_df['is_weekend'] = train_df['weekday'].isin([5, 6]) * 1

In [87]:
test_df["weekday"] = test_df.period_dt.dt.weekday
test_df['is_weekend'] = test_df['weekday'].isin([5, 6]) * 1

In [88]:
train_df = train_df.sort_values(by=['product_id', 'location_id', 'period_dt'])
test_df = test_df.sort_values(by=['product_id', 'location_id', 'period_dt'])

product_launch_dates = train_df.groupby(['product_id', 'location_id'])['period_dt'].min().reset_index()
product_launch_dates.columns = ['product_id', 'location_id', 'launch_date']

train_df = train_df.merge(product_launch_dates, on=['product_id', 'location_id'], how='left')
test_df = test_df.merge(product_launch_dates, on=['product_id', 'location_id'], how='left')

train_df['days_since_launch'] = (train_df['period_dt'] - train_df['launch_date']).dt.days
test_df['days_since_launch'] = (test_df['period_dt'] - test_df['launch_date']).dt.days

In [89]:
train_df = train_df[~((train_df['launch_date'] >= '2016-12-01') & (train_df['launch_date'] <= '2017-01-01'))]

train_df['early_life_cycle'] = train_df['days_since_launch'].between(0, 28)  # 4 weeks as 28 days


cutoff_date = '2017-12-30'

train_df['early_life_cycle_end'] = train_df['launch_date'] + pd.to_timedelta(28, unit='d')


cutoff_date = pd.to_datetime('2017-12-30')
old_products = train_df[train_df['early_life_cycle_end'] < cutoff_date]

<ipython-input-89-b625147108fa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['early_life_cycle'] = train_df['days_since_launch'].between(0, 28)  # 4 weeks as 28 days
<ipython-input-89-b625147108fa>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['early_life_cycle_end'] = train_df['launch_date'] + pd.to_timedelta(28, unit='d')


In [90]:

old_products = old_products.dropna()


In [91]:
old_products

,period_dt,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,...,day,season_Spring,season_Summer,season_Winter,weekday,is_weekend,launch_date,days_since_launch,early_life_cycle,early_life_cycle_end
1,2017-05-08,0.0,474.5,949.0,9988,0.0,0.0,10,189,189,...,8,1,0,0,0,0,2017-05-08,0,True,2017-06-05
2,2017-05-15,0.0,474.5,949.0,9988,0.0,0.0,10,189,189,...,15,1,0,0,0,0,2017-05-08,7,True,2017-06-05
3,2017-05-22,0.0,474.5,949.0,9988,0.0,0.0,10,189,189,...,22,1,0,0,0,0,2017-05-08,14,True,2017-06-05
4,2017-05-29,0.0,474.5,949.0,9988,1.0,0.0,10,189,189,...,29,1,0,0,0,0,2017-05-08,21,True,2017-06-05
5,2017-06-05,0.0,474.5,949.0,9988,1.0,0.0,10,189,189,...,5,0,1,0,0,0,2017-05-08,28,True,2017-06-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508174,2018-12-03,5.0,1749.5,3499.0,113176,0.0,0.0,10,203,203,...,3,0,0,1,0,0,2017-11-27,371,False,2017-12-25
508175,2018-12-10,5.0,1749.5,3499.0,113176,0.0,0.0,10,203,203,...,10,0,0,1,0,0,2017-11-27,378,False,2017-12-25
508176,2018-12-17,5.0,1749.5,3499.0,113176,0.0,0.0,10,203,203,...,17,0,0,1,0,0,2017-11-27,385,False,2017-12-25
508177,2018-12-24,5.0,1749.5,3499.0,113176,0.0,0.0,10,203,203,...,24,0,0,1,0,0,2017-11-27,392,False,2017-12-25


In [92]:
test_df

,product_id,location_id,period_dt,id,year,month,day,season_Spring,season_Summer,season_Winter,weekday,is_weekend,launch_date,days_since_launch
0,19474,425,2019-04-08,0,2019,4,8,1,0,0,0,0,2019-04-08,0
1,19581,862,2019-04-08,1772,2019,4,8,1,0,0,0,0,2019-04-08,0
2,19581,862,2019-04-15,1773,2019,4,15,1,0,0,0,0,2019-04-08,7
3,19581,862,2019-04-22,1774,2019,4,22,1,0,0,0,0,2019-04-08,14
4,19581,862,2019-04-29,1775,2019,4,29,1,0,0,0,0,2019-04-08,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34102,148464,1326,2019-12-23,550139,2019,12,23,0,0,1,0,0,2019-12-16,7
34103,148464,1326,2019-12-30,550140,2019,12,30,0,0,1,0,0,2019-12-16,14
34104,148464,1380,2019-12-23,550141,2019,12,23,0,0,1,0,0,2019-12-23,0
34105,148464,1380,2019-12-30,550142,2019,12,30,0,0,1,0,0,2019-12-23,7


In [93]:
old_products.sort_values(by=['product_id', 'location_id', 'period_dt'], inplace=True)

## Lag

### lag in test

In [99]:
train_df = train_df.drop(columns = 'id')

In [100]:
combined_df = pd.concat([test_df, train_df], ignore_index=True)

In [101]:
combined_df.sort_values('id')

,product_id,location_id,period_dt,id,year,month,day,season_Spring,season_Summer,season_Winter,...,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,early_life_cycle,early_life_cycle_end
0,19474,425,2019-04-08,0.0,2019,4,8,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,19581,862,2019-04-08,1772.0,2019,4,8,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,19581,862,2019-04-15,1773.0,2019,4,15,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,19581,862,2019-04-22,1774.0,2019,4,22,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,19581,862,2019-04-29,1775.0,2019,4,29,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505123,148464,1326,2019-12-23,NaN,2019,12,23,0,0,1,...,1399.0,113176.0,1.0,0.0,10.0,203.0,203.0,NaN,True,2020-01-13
505124,148464,1326,2019-12-30,NaN,2019,12,30,0,0,1,...,1399.0,113176.0,1.0,0.0,10.0,203.0,203.0,NaN,True,2020-01-13
505125,148464,1380,2019-12-23,NaN,2019,12,23,0,0,1,...,1399.0,113176.0,1.0,0.0,10.0,219.0,219.0,NaN,True,2020-01-20
505126,148464,1380,2019-12-30,NaN,2019,12,30,0,0,1,...,1399.0,113176.0,1.0,0.0,10.0,219.0,219.0,NaN,True,2020-01-20


In [102]:
combined_df.sort_values(by=['product_id', 'location_id', 'period_dt'], inplace=True)

In [103]:
data_c = combined_df

In [104]:
data_c

,product_id,location_id,period_dt,id,year,month,day,season_Spring,season_Summer,season_Winter,...,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,early_life_cycle,early_life_cycle_end
0,19474,425,2019-04-08,0.0,2019,4,8,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
34107,19474,425,2019-04-08,NaN,2019,4,8,1,0,0,...,1049.0,9988.0,1.0,0.0,10.0,203.0,203.0,NaN,True,2019-05-06
34108,19474,453,2017-05-08,NaN,2017,5,8,1,0,0,...,949.0,9988.0,0.0,0.0,10.0,189.0,189.0,1.016438,True,2017-06-05
34109,19474,453,2017-05-15,NaN,2017,5,15,1,0,0,...,949.0,9988.0,0.0,0.0,10.0,189.0,189.0,0.019178,True,2017-06-05
34110,19474,453,2017-05-22,NaN,2017,5,22,1,0,0,...,949.0,9988.0,0.0,0.0,10.0,189.0,189.0,0.019178,True,2017-06-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505125,148464,1380,2019-12-23,NaN,2019,12,23,0,0,1,...,1399.0,113176.0,1.0,0.0,10.0,219.0,219.0,NaN,True,2020-01-20
34105,148464,1380,2019-12-30,550142.0,2019,12,30,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
505126,148464,1380,2019-12-30,NaN,2019,12,30,0,0,1,...,1399.0,113176.0,1.0,0.0,10.0,219.0,219.0,NaN,True,2020-01-20
34106,155459,1328,2019-12-30,550143.0,2019,12,30,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [105]:
lags = {
    '52_week': 52,
    '104_weeks': 104  # Roughly one month, adjust according to your definition of a month
}

# Create lagged features
for lag_name, lag_periods in lags.items():
    data_c[f'lag_{lag_name}'] = data_c['demand'].shift(lag_periods)

# Drop rows with NaN if necessary
#data.dropna(inplace=True)

In [106]:
data_c

,product_id,location_id,period_dt,id,year,month,day,season_Spring,season_Summer,season_Winter,...,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,early_life_cycle,early_life_cycle_end,lag_52_week,lag_104_weeks
0,19474,425,2019-04-08,0.0,2019,4,8,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
34107,19474,425,2019-04-08,NaN,2019,4,8,1,0,0,...,1.0,0.0,10.0,203.0,203.0,NaN,True,2019-05-06,NaN,NaN
34108,19474,453,2017-05-08,NaN,2017,5,8,1,0,0,...,0.0,0.0,10.0,189.0,189.0,1.016438,True,2017-06-05,NaN,NaN
34109,19474,453,2017-05-15,NaN,2017,5,15,1,0,0,...,0.0,0.0,10.0,189.0,189.0,0.019178,True,2017-06-05,NaN,NaN
34110,19474,453,2017-05-22,NaN,2017,5,22,1,0,0,...,0.0,0.0,10.0,189.0,189.0,0.019178,True,2017-06-05,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505125,148464,1380,2019-12-23,NaN,2019,12,23,0,0,1,...,1.0,0.0,10.0,219.0,219.0,NaN,True,2020-01-20,NaN,NaN
34105,148464,1380,2019-12-30,550142.0,2019,12,30,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
505126,148464,1380,2019-12-30,NaN,2019,12,30,0,0,1,...,1.0,0.0,10.0,219.0,219.0,NaN,True,2020-01-20,NaN,NaN
34106,155459,1328,2019-12-30,550143.0,2019,12,30,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN


In [107]:
data_c = data_c.dropna(subset='id')

In [108]:
data_c

,product_id,location_id,period_dt,id,year,month,day,season_Spring,season_Summer,season_Winter,...,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,demand,early_life_cycle,early_life_cycle_end,lag_52_week,lag_104_weeks
0,19474,425,2019-04-08,0.0,2019,4,8,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,19581,862,2019-04-08,1772.0,2019,4,8,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,0.0
2,19581,862,2019-04-15,1773.0,2019,4,15,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,0.0
3,19581,862,2019-04-22,1774.0,2019,4,22,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,0.0
4,19581,862,2019-04-29,1775.0,2019,4,29,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34102,148464,1326,2019-12-23,550139.0,2019,12,23,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
34103,148464,1326,2019-12-30,550140.0,2019,12,30,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
34104,148464,1380,2019-12-23,550141.0,2019,12,23,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
34105,148464,1380,2019-12-30,550142.0,2019,12,30,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN


In [109]:
columns_to_add = ['lag_52_week', 'lag_104_weeks']

# Merge dataframes using a left join
result_df = pd.merge(test_df, data_c[['product_id', 'location_id', 'period_dt'] + columns_to_add],
                     on=['product_id', 'location_id', 'period_dt'],
                     how='left')


In [110]:
result_df.sort_values('lag_52_week')

,product_id,location_id,period_dt,id,year,month,day,season_Spring,season_Summer,season_Winter,weekday,is_weekend,launch_date,days_since_launch,lag_52_week,lag_104_weeks
4776,77666,504,2019-09-23,270206,2019,9,23,0,0,0,0,0,2019-05-27,119,0.0,0.0
7191,92961,1380,2019-06-10,382390,2019,6,10,0,1,0,0,0,2019-06-10,0,0.0,NaN
7189,92961,1162,2019-08-26,381827,2019,8,26,0,1,0,0,0,2019-03-04,175,0.0,0.0
7164,92961,1162,2019-03-04,381802,2019,3,4,1,0,0,0,0,2019-03-04,0,0.0,0.0
7163,92961,1034,2019-12-16,381797,2019,12,16,0,0,1,0,0,2019-07-15,154,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34102,148464,1326,2019-12-23,550139,2019,12,23,0,0,1,0,0,2019-12-16,7,NaN,NaN
34103,148464,1326,2019-12-30,550140,2019,12,30,0,0,1,0,0,2019-12-16,14,NaN,NaN
34104,148464,1380,2019-12-23,550141,2019,12,23,0,0,1,0,0,2019-12-23,0,NaN,NaN
34105,148464,1380,2019-12-30,550142,2019,12,30,0,0,1,0,0,2019-12-23,7,NaN,NaN


In [111]:
result_df = result_df.drop(columns = ['launch_date', 'period_dt'], axis = 0 )

In [112]:
result_df , id = result_df.drop(columns = ['id'], axis = 0 ), result_df['id']

In [113]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34107 entries, 0 to 34106
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_id         34107 non-null  int64  
 1   location_id        34107 non-null  int64  
 2   year               34107 non-null  int32  
 3   month              34107 non-null  int32  
 4   day                34107 non-null  int32  
 5   season_Spring      34107 non-null  int64  
 6   season_Summer      34107 non-null  int64  
 7   season_Winter      34107 non-null  int64  
 8   weekday            34107 non-null  int32  
 9   is_weekend         34107 non-null  int64  
 10  days_since_launch  34107 non-null  int64  
 11  lag_52_week        2473 non-null   float64
 12  lag_104_weeks      6943 non-null   float64
dtypes: float64(2), int32(4), int64(7)
memory usage: 2.9 MB


### в трэйне

In [114]:
data = old_products

In [115]:
data.head()

,period_dt,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,...,day,season_Spring,season_Summer,season_Winter,weekday,is_weekend,launch_date,days_since_launch,early_life_cycle,early_life_cycle_end
1,2017-05-08,0.0,474.5,949.0,9988,0.0,0.0,10,189,189,...,8,1,0,0,0,0,2017-05-08,0,True,2017-06-05
2,2017-05-15,0.0,474.5,949.0,9988,0.0,0.0,10,189,189,...,15,1,0,0,0,0,2017-05-08,7,True,2017-06-05
3,2017-05-22,0.0,474.5,949.0,9988,0.0,0.0,10,189,189,...,22,1,0,0,0,0,2017-05-08,14,True,2017-06-05
4,2017-05-29,0.0,474.5,949.0,9988,1.0,0.0,10,189,189,...,29,1,0,0,0,0,2017-05-08,21,True,2017-06-05
5,2017-06-05,0.0,474.5,949.0,9988,1.0,0.0,10,189,189,...,5,0,1,0,0,0,2017-05-08,28,True,2017-06-05


In [116]:
lags = {
    '52_week': 52,
    '104_weeks': 104  # Roughly one month, adjust according to your definition of a month
}

# Create lagged features
for lag_name, lag_periods in lags.items():
    data[f'lag_{lag_name}'] = data['demand'].shift(lag_periods)

# Drop rows with NaN if necessary
data.dropna(inplace=True)

In [117]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183045 entries, 175 to 508178
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   period_dt               183045 non-null  datetime64[ns]
 1   NUM_CONSULTANT          183045 non-null  float64       
 2   PRICE_AFTER_DISC        183045 non-null  float64       
 3   PRICE_REGULAR           183045 non-null  float64       
 4   PRODUCT_LVL_RK6         183045 non-null  int64         
 5   PROMO1_FLAG             183045 non-null  float64       
 6   PROMO2_FLAG             183045 non-null  float64       
 7   STORE_LOCATION_LVL_RK2  183045 non-null  int64         
 8   STORE_LOCATION_LVL_RK3  183045 non-null  int64         
 9   STORE_LOCATION_LVL_RK4  183045 non-null  int64         
 10  demand                  183045 non-null  float64       
 11  location_id             183045 non-null  int64         
 12  product_id              183045 no

In [118]:
data.head()

,period_dt,NUM_CONSULTANT,PRICE_AFTER_DISC,PRICE_REGULAR,PRODUCT_LVL_RK6,PROMO1_FLAG,PROMO2_FLAG,STORE_LOCATION_LVL_RK2,STORE_LOCATION_LVL_RK3,STORE_LOCATION_LVL_RK4,...,season_Summer,season_Winter,weekday,is_weekend,launch_date,days_since_launch,early_life_cycle,early_life_cycle_end,lag_52_week,lag_104_weeks
175,2017-01-30,0.0,350.0,350.0,9988,1.0,0.0,10,28,28,...,0,1,0,0,2017-01-02,28,True,2017-01-30,1.0,1.016438
176,2017-02-06,0.0,350.0,350.0,9988,1.0,0.0,10,28,28,...,0,1,0,0,2017-01-02,35,False,2017-01-30,0.0,0.019178
177,2017-02-13,0.0,350.0,350.0,9988,1.0,0.0,10,28,28,...,0,1,0,0,2017-01-02,42,False,2017-01-30,0.0,0.019178
178,2017-02-20,0.0,350.0,350.0,9988,1.0,0.0,10,28,28,...,0,1,0,0,2017-01-02,49,False,2017-01-30,0.0,0.016438
179,2017-02-27,0.0,350.0,350.0,9988,1.0,0.0,10,28,28,...,0,1,0,0,2017-01-02,56,False,2017-01-30,0.0,0.000000


In [119]:
X_train = data[['product_id', 'location_id', 'days_since_launch', 'lag_52_week', 'lag_104_weeks',
                'year', 'month', 'day', 'season_Spring', 'season_Summer', 'season_Winter', 'weekday', 'is_weekend']]
y_train = data['demand']


In [43]:
X_train = data.drop(columns = ['period_dt','launch_date'], axis =0)
y_train = data['demand']


In [120]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183045 entries, 175 to 508178
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   product_id         183045 non-null  int64  
 1   location_id        183045 non-null  int64  
 2   days_since_launch  183045 non-null  int64  
 3   lag_52_week        183045 non-null  float64
 4   lag_104_weeks      183045 non-null  float64
 5   year               183045 non-null  int32  
 6   month              183045 non-null  int32  
 7   day                183045 non-null  int32  
 8   season_Spring      183045 non-null  int64  
 9   season_Summer      183045 non-null  int64  
 10  season_Winter      183045 non-null  int64  
 11  weekday            183045 non-null  int32  
 12  is_weekend         183045 non-null  int64  
dtypes: float64(2), int32(4), int64(7)
memory usage: 16.8 MB


In [121]:
test_df.sort_values('period_dt')

,product_id,location_id,period_dt,id,year,month,day,season_Spring,season_Summer,season_Winter,weekday,is_weekend,launch_date,days_since_launch
10632,95413,764,2019-01-07,443773,2019,1,7,0,0,1,0,0,2019-01-07,0
22839,104312,1363,2019-01-07,519253,2019,1,7,0,0,1,0,0,2019-01-07,0
11930,97345,1347,2019-01-07,463473,2019,1,7,0,0,1,0,0,2019-01-07,0
12390,99326,1380,2019-01-07,479305,2019,1,7,0,0,1,0,0,2019-01-07,0
5443,78386,1281,2019-01-07,304840,2019,1,7,0,0,1,0,0,2019-01-07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8871,95321,557,2019-12-30,420255,2019,12,30,0,0,1,0,0,2019-01-14,350
23174,104364,1191,2019-12-30,522387,2019,12,30,0,0,1,0,0,2019-06-24,189
29892,136434,1328,2019-12-30,545929,2019,12,30,0,0,1,0,0,2019-07-01,182
23254,106750,1326,2019-12-30,525099,2019,12,30,0,0,1,0,0,2019-04-15,259


In [47]:
test_df

,product_id,location_id,period_dt,id,year,month,day,season_Spring,season_Summer,season_Winter,weekday,is_weekend,launch_date,days_since_launch
0,19474,425,2019-04-08,0,2019,4,8,1,0,0,0,0,2019-04-08,0
1,19581,862,2019-04-08,1772,2019,4,8,1,0,0,0,0,2019-04-08,0
2,19581,862,2019-04-15,1773,2019,4,15,1,0,0,0,0,2019-04-08,7
3,19581,862,2019-04-22,1774,2019,4,22,1,0,0,0,0,2019-04-08,14
4,19581,862,2019-04-29,1775,2019,4,29,1,0,0,0,0,2019-04-08,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34102,148464,1326,2019-12-23,550139,2019,12,23,0,0,1,0,0,2019-12-16,7
34103,148464,1326,2019-12-30,550140,2019,12,30,0,0,1,0,0,2019-12-16,14
34104,148464,1380,2019-12-23,550141,2019,12,23,0,0,1,0,0,2019-12-23,0
34105,148464,1380,2019-12-30,550142,2019,12,30,0,0,1,0,0,2019-12-23,7


## Model

In [122]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183045 entries, 175 to 508178
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   product_id         183045 non-null  int64  
 1   location_id        183045 non-null  int64  
 2   days_since_launch  183045 non-null  int64  
 3   lag_52_week        183045 non-null  float64
 4   lag_104_weeks      183045 non-null  float64
 5   year               183045 non-null  int32  
 6   month              183045 non-null  int32  
 7   day                183045 non-null  int32  
 8   season_Spring      183045 non-null  int64  
 9   season_Summer      183045 non-null  int64  
 10  season_Winter      183045 non-null  int64  
 11  weekday            183045 non-null  int32  
 12  is_weekend         183045 non-null  int64  
dtypes: float64(2), int32(4), int64(7)
memory usage: 16.8 MB


In [123]:
from sklearn.ensemble import GradientBoostingRegressor

regressor = GradientBoostingRegressor(
    max_depth=11,
    n_estimators=420,
    learning_rate=0.2,
    random_state=1,
    min_samples_leaf=11,
    min_samples_split=2,
    loss='absolute_error'
)
regressor.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.2, loss='absolute_error',
                          max_depth=11, min_samples_leaf=11, n_estimators=420,
                          random_state=1)

In [124]:
result_df = result_df.fillna(0)

In [125]:
result_df

,product_id,location_id,year,month,day,season_Spring,season_Summer,season_Winter,weekday,is_weekend,days_since_launch,lag_52_week,lag_104_weeks
0,19474,425,2019,4,8,1,0,0,0,0,0,0.0,0.0
1,19581,862,2019,4,8,1,0,0,0,0,0,0.0,0.0
2,19581,862,2019,4,15,1,0,0,0,0,7,0.0,0.0
3,19581,862,2019,4,22,1,0,0,0,0,14,0.0,0.0
4,19581,862,2019,4,29,1,0,0,0,0,21,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34102,148464,1326,2019,12,23,0,0,1,0,0,7,0.0,0.0
34103,148464,1326,2019,12,30,0,0,1,0,0,14,0.0,0.0
34104,148464,1380,2019,12,23,0,0,1,0,0,0,0.0,0.0
34105,148464,1380,2019,12,30,0,0,1,0,0,7,0.0,0.0


In [126]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183045 entries, 175 to 508178
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   product_id         183045 non-null  int64  
 1   location_id        183045 non-null  int64  
 2   days_since_launch  183045 non-null  int64  
 3   lag_52_week        183045 non-null  float64
 4   lag_104_weeks      183045 non-null  float64
 5   year               183045 non-null  int32  
 6   month              183045 non-null  int32  
 7   day                183045 non-null  int32  
 8   season_Spring      183045 non-null  int64  
 9   season_Summer      183045 non-null  int64  
 10  season_Winter      183045 non-null  int64  
 11  weekday            183045 non-null  int32  
 12  is_weekend         183045 non-null  int64  
dtypes: float64(2), int32(4), int64(7)
memory usage: 16.8 MB


In [127]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34107 entries, 0 to 34106
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_id         34107 non-null  int64  
 1   location_id        34107 non-null  int64  
 2   year               34107 non-null  int32  
 3   month              34107 non-null  int32  
 4   day                34107 non-null  int32  
 5   season_Spring      34107 non-null  int64  
 6   season_Summer      34107 non-null  int64  
 7   season_Winter      34107 non-null  int64  
 8   weekday            34107 non-null  int32  
 9   is_weekend         34107 non-null  int64  
 10  days_since_launch  34107 non-null  int64  
 11  lag_52_week        34107 non-null  float64
 12  lag_104_weeks      34107 non-null  float64
dtypes: float64(2), int32(4), int64(7)
memory usage: 2.9 MB


In [128]:
result_df = result_df[['product_id','location_id', 'days_since_launch', 'lag_52_week', 'lag_104_weeks', 'year', 'month', 'day', 'season_Spring', 'season_Summer', 'season_Winter', 'weekday', 'is_weekend']]

In [129]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34107 entries, 0 to 34106
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   product_id         34107 non-null  int64  
 1   location_id        34107 non-null  int64  
 2   days_since_launch  34107 non-null  int64  
 3   lag_52_week        34107 non-null  float64
 4   lag_104_weeks      34107 non-null  float64
 5   year               34107 non-null  int32  
 6   month              34107 non-null  int32  
 7   day                34107 non-null  int32  
 8   season_Spring      34107 non-null  int64  
 9   season_Summer      34107 non-null  int64  
 10  season_Winter      34107 non-null  int64  
 11  weekday            34107 non-null  int32  
 12  is_weekend         34107 non-null  int64  
dtypes: float64(2), int32(4), int64(7)
memory usage: 2.9 MB


In [130]:
a = regressor.predict(result_df)

In [131]:
id

0             0
1          1772
2          1773
3          1774
4          1775
          ...  
34102    550139
34103    550140
34104    550141
34105    550142
34106    550143
Name: id, Length: 34107, dtype: int64

In [132]:
result_df = pd.DataFrame({ 'id': id, 'demand': a})
#result_df = result_df.join(id, on='id', how='left', lsuffix='predicted')


In [133]:
result_df

,id,demand
0,0,1.000000
1,1772,1.000000
2,1773,0.000000
3,1774,0.000000
4,1775,0.000000
...,...,...
34102,550139,1.175145
34103,550140,1.175145
34104,550141,1.000000
34105,550142,1.175145


In [135]:
result_df.to_csv('predi.csv', index = False)

# Zero

In [185]:
# Binarize the target for classifier
y_class = (y_train > 0).astype(int)

In [186]:
# Separate majority and minority classes
train_majority = data[data['demand'] == 0]
train_minority = data[data['demand'] > 0]

In [188]:
from sklearn.utils import resample
# Upsample minority class
train_minority_upsampled = resample(train_minority,
                                    replace=True,     # sample with replacement
                                    n_samples=len(train_majority),    # to match majority class
                                    random_state=42) # reproducible results

In [201]:
train_upsampled = pd.concat([train_majority, train_minority_upsampled])

In [202]:
train_upsampled = train_upsampled[['product_id', 'location_id', 'days_since_launch', 'lag_1_week', 'lag_2_weeks', 'lag_1_month', 'demand']]

In [203]:
# Split the data into features and target
X_upsampled = train_upsampled.drop(columns=['demand'])
y_upsampled = train_upsampled['demand']

In [204]:
# Binarize the target for classifier
y_class_upsampled = (y_upsampled > 0).astype(int)

In [206]:
y_upsampled

5         0.000000
6         0.000000
7         0.000000
8         0.000000
18        0.000000
            ...   
310638    0.065041
94057     0.020979
365252    1.000000
212086    0.067961
263636    0.433333
Name: demand, Length: 297254, dtype: float64

In [207]:
from sklearn.ensemble import GradientBoostingClassifier
# Train a Gradient Boosting Classifier
classifier_gb = GradientBoostingClassifier(random_state=42)
classifier_gb.fit(X_upsampled, y_class_upsampled)


GradientBoostingClassifier(random_state=42)

In [196]:
X_upsampled = X_upsampled

<class 'pandas.core.frame.DataFrame'>
Index: 297254 entries, 5 to 263636
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   NUM_CONSULTANT          297254 non-null  float64       
 1   PRICE_AFTER_DISC        297254 non-null  float64       
 2   PRICE_REGULAR           297254 non-null  float64       
 3   PRODUCT_LVL_RK6         297254 non-null  int64         
 4   PROMO1_FLAG             297254 non-null  float64       
 5   PROMO2_FLAG             297254 non-null  float64       
 6   STORE_LOCATION_LVL_RK2  297254 non-null  int64         
 7   STORE_LOCATION_LVL_RK3  297254 non-null  int64         
 8   STORE_LOCATION_LVL_RK4  297254 non-null  int64         
 9   location_id             297254 non-null  int64         
 10  product_id              297254 non-null  int64         
 11  launch_date             297254 non-null  datetime64[ns]
 12  days_since_launch       297254 non-

In [210]:
x = data[['product_id', 'location_id', 'days_since_launch', 'lag_1_week', 'lag_2_weeks', 'lag_1_month', 'demand']]

In [212]:
from sklearn.ensemble import RandomForestRegressor
# Filter the data for non-zero demand
non_zero_df = x[x['demand'] > 0]

X_reg = non_zero_df.drop(columns=['demand'])
y_reg = non_zero_df['demand']

# Train the regressor
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_reg, y_reg)

RandomForestRegressor(random_state=42)

In [213]:
# Ensure the test set has the same features as the training set
required_features = X_train.columns
result_df = result_df.reindex(columns=required_features, fill_value=0)

# Predict using the Gradient Boosting Classifier
test_pred_class_gb = classifier_gb.predict(result_df)

# Initialize the final predictions with zeros
test_predictions = [0] * len(result_df)

# Predict using the regressor for non-zero cases
non_zero_indices = [i for i, val in enumerate(test_pred_class_gb) if val == 1]
X_test_non_zero = result_df.iloc[non_zero_indices]
test_pred_reg = regressor.predict(X_test_non_zero)

# Place the regressor predictions in the final predictions
for idx, pred in zip(non_zero_indices, test_pred_reg):
    test_predictions[idx] = pred

# Prepare the final submission dataframe
submission_df = result_df.copy()
submission_df['demand'] = test_predictions

print(submission_df.head())

   product_id  location_id  days_since_launch  lag_1_week  lag_2_weeks  \
0           0            0                  0           0            0   
1           0            0                  0           0            0   
2           0            0                  0           0            0   
3           0            0                  0           0            0   
4           0            0                  0           0            0   

   lag_1_month    demand  
0            0  0.748203  
1            0  0.748203  
2            0  0.748203  
3            0  0.748203  
4            0  0.748203  


In [214]:
submission_df = submission_df.drop(columns=['product_id', 'location_id', 'days_since_launch', 'lag_1_week', 'lag_2_weeks', 'lag_1_month'])

In [217]:
a = submission_df['demand']

In [215]:
print(submission_df.head())

     demand
0  0.748203
1  0.748203
2  0.748203
3  0.748203
4  0.748203


In [218]:
result_df = pd.DataFrame({ 'id': id, 'demand': a})
#result_df = result_df.join(id, on='id', how='left', lsuffix='predicted')


In [220]:
result_df.to_csv('predi.csv', index = False)

In [ ]:
submission_df.to_csv('/content/drive/My Drive/hse-dab-lt-naf-24/predicted_demand.csv', index=False)